In [677]:
from ast import literal_eval

import numpy as np
import pandas as pd

In [678]:
movies = pd.read_csv("processed_data/movies.csv")

In [679]:
movies.columns

Index(['Unnamed: 0', 'genres', 'id', 'imdb_id', 'original_language',
       'overview', 'popularity', 'tagline', 'title', 'vote_average',
       'vote_count', 'year', 'description', 'movie_id', 'cast', 'keywords',
       'director'],
      dtype='object')

In [680]:
movies['description'] + movies['keywords'] + movies['director'] + movies['cast']

0      minions stuart, kevin and bob are recruited by...
1      an amazon princess comes to the world of man t...
2      a live-action adaptation of disney's version o...
3      after being coerced into working for a crime b...
4      the special bond that develops between plus-si...
                             ...                        
495    this third film in the series follows a group ...
496    as kevin flynn searches for proof that he inve...
497    in a gritty and alternate 1985 the glory days ...
498    a group of young journalists investigate a cul...
499    set in 1879 paris. an orphan girl dreams of be...
Length: 500, dtype: object

In [681]:
def concat_list_data_to_single_dataframe(dataframe, column, new_column_name=None):
    new_column_name = new_column_name if new_column_name is not None else column
    serie = dataframe[column].apply(literal_eval).apply(lambda x: [str.lower("".join(i.split(" "))).replace(".", "").replace("-", "").replace("'", "") for i in x])
    serie = serie.apply(lambda x: " ".join(x))
    serie_frame = pd.concat([pd.DataFrame(i, columns=[new_column_name]) 
                                for i in serie.apply(lambda x: [i for i in x.split(" ")])
                               ], ignore_index=True)
    serie_frame.drop(inplace=True, index=serie_frame.index[serie_frame[new_column_name].duplicated()])
    serie_frame.drop(inplace=True, index=serie_frame.index[serie_frame[new_column_name] == ""])
    return serie_frame

In [682]:
def remove_duplicates(df, columnname):
    df.drop(inplace=True, index=df.index[df[columnname].duplicated()])

In [683]:
keywords_df = concat_list_data_to_single_dataframe(movies, "keywords", "keyword")
print(keywords_df.shape)

(2721, 1)


In [684]:
keywords_df

,keyword
0,assistant
1,aftercreditsstinger
2,duringcreditsstinger
3,evilmastermind
4,minions
...,...
5382,deathofsuperhero
5383,americanpresident
5397,opera
5402,statueofliberty


In [685]:
cast_df = concat_list_data_to_single_dataframe(movies, "cast")
print(cast_df.shape)

(1243, 1)


In [686]:
cast_df

,cast
0,sandrabullock
1,jonhamm
2,michaelkeaton
3,allisonjanney
4,galgadot
...,...
1990,aaronashmore
1991,meghanheffern
1992,benlewis
1995,carlyraejepsen


In [687]:
directors = movies.director.apply(lambda x: str.lower("".join(x.split(" "))).replace(".", "").replace("-", "").replace("'", ""))
directors.drop(inplace=True, index=directors.index[directors.duplicated()])
print(directors.shape)

(331,)


In [688]:
genres = concat_list_data_to_single_dataframe(movies, "genres", "genre")

In [689]:
print(genres.shape)
genres

(18, 1)


,genre
0,family
1,animation
2,adventure
3,comedy
4,action
6,fantasy
9,romance
11,crime
23,sciencefiction
29,thriller


In [690]:
df = pd.concat([genres.genre, directors, cast_df.cast, keywords_df.keyword], ignore_index=True)
df.name = 0
df

0                  family
1               animation
2               adventure
3                  comedy
4                  action
              ...        
4308     deathofsuperhero
4309    americanpresident
4310                opera
4311      statueofliberty
4312     eiffeltowerparis
Name: 0, Length: 4313, dtype: object

In [691]:
movies.keywords = movies.keywords.apply(literal_eval).apply(
                        lambda x: [str.lower("".join(i.split(" "))).replace(".", "").replace("-", "").replace("'", "") for i in x]
                    ).apply(lambda x: " ".join(x))
movies.keywords

0      assistant aftercreditsstinger duringcreditssti...
1      dccomics hero greekmythology island worldwari ...
2      france magic castle fairytale musical curse an...
3      robbery atlanta music crimeboss romance tinnit...
4      brotherbrotherrelationship hero talent revenge...
                             ...                        
495    diving dolphin florida sequel scubadiving swim...
496    videogame hacker virtualreality dystopia compu...
497    dccomics secretidentity massmurder retirement ...
498    journalist coffin diary village supernatural c...
499    paris flying inventor dance opera musical life...
Name: keywords, Length: 500, dtype: object

In [692]:
movies.cast = movies.cast.apply(literal_eval).apply(
                        lambda x: [str.lower("".join(i.split(" "))) for i in x]
                    ).apply(lambda x: " ".join(x))
movies.cast

0      sandrabullock jonhamm michaelkeaton allisonjanney
1             galgadot chrispine robinwright dannyhuston
2             emmawatson danstevens lukeevans kevinkline
3            anselelgort lilyjames kevinspacey jamiefoxx
4          scottadsit ryanpotter danielhenney t.j.miller
                             ...                        
495    dennisquaid bessarmstrong simonmaccorkindale l...
496    jeffbridges bruceboxleitner davidwarner cindym...
497    malinåkerman billycrudup matthewgoode jackieea...
498     cindysampson aaronashmore meghanheffern benlewis
499    ellefanning danedehaan carlyraejepsen maddiezi...
Name: cast, Length: 500, dtype: object

In [693]:
movies.genres = movies.genres.apply(literal_eval).apply(
                        lambda x: [str.lower("".join(i.split(" "))) for i in x]
                    ).apply(lambda x: " ".join(x))
movies.genres

0             family animation adventure comedy
1                      action adventure fantasy
2                        family fantasy romance
3                                  action crime
4      adventure family animation action comedy
                         ...                   
495                             thriller horror
496             sciencefiction action adventure
497               action mystery sciencefiction
498                                      horror
499           animation family adventure comedy
Name: genres, Length: 500, dtype: object

In [694]:
movies.director = movies.director.apply(lambda x: str.lower("".join(x.split(" "))).replace(".", "").replace("-", "").replace("'", ""))
movies.director

0            kylebalda
1         pattyjenkins
2           billcondon
3          edgarwright
4        chriswilliams
            ...       
495           joealves
496    stevenlisberger
497         zacksnyder
498          jonknautz
499         ericsummer
Name: director, Length: 500, dtype: object

In [695]:
concat = movies.director + " " + movies.genres + " " + movies.cast + " " + movies.keywords
concat

0      kylebalda family animation adventure comedy sa...
1      pattyjenkins action adventure fantasy galgadot...
2      billcondon family fantasy romance emmawatson d...
3      edgarwright action crime anselelgort lilyjames...
4      chriswilliams adventure family animation actio...
                             ...                        
495    joealves thriller horror dennisquaid bessarmst...
496    stevenlisberger sciencefiction action adventur...
497    zacksnyder action mystery sciencefiction malin...
498    jonknautz horror cindysampson aaronashmore meg...
499    ericsummer animation family adventure comedy e...
Length: 500, dtype: object

In [696]:
concat.str.contains("action").sum()

210

In [697]:
concat.str.contains("stevenspielbergstevenspielbergroyscheider").sum()

0

In [698]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [699]:
tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words='english')
data_for_tfidf = movies.description
data_for_tfidf += " " + movies.director + " " + movies.director
data_for_tfidf += " " + movies.cast + " " + movies.cast + " " + movies.cast
data_for_tfidf += " " + movies.keywords + " " + movies.keywords + " " + movies.keywords
data_for_tfidf += " " + movies.genres + " " + movies.genres + " " + movies.genres + " " + movies.genres + " " + movies.genres
tfidf_matrix = tf.fit_transform(data_for_tfidf)

In [700]:
tfidf_matrix.shape

(500, 9265)

In [701]:
print(tfidf_matrix[0])

  (0, 1698)	0.20091610687341463
  (0, 227)	0.15176385894997932
  (0, 462)	0.24322778035354126
  (0, 3001)	0.18427983601104442
  (0, 90)	0.15480738883908798
  (0, 2858)	0.2999827985026596
  (0, 2566)	0.16108784705874768
  (0, 245)	0.1645820769296956
  (0, 621)	0.28133738978508877
  (0, 352)	0.2999827985026596
  (0, 5439)	0.2578469528817384
  (0, 4501)	0.28133738978508877
  (0, 7210)	0.28133738978508877
  (0, 4736)	0.18755825985672586
  (0, 1044)	0.09999426616755319
  (0, 718)	0.07252943877199614
  (0, 3842)	0.05900381385075643
  (0, 3599)	0.0859489842939128
  (0, 9167)	0.03842949237180756
  (0, 6340)	0.071302511169111
  (0, 3711)	0.09377912992836293
  (0, 3789)	0.09999426616755319
  (0, 3970)	0.07693914811098924
  (0, 4211)	0.08079139921572145
  (0, 360)	0.09999426616755319
  (0, 8878)	0.07252943877199614
  (0, 8051)	0.0753241387157294
  (0, 6050)	0.09999426616755319
  (0, 7261)	0.09377912992836293
  (0, 6755)	0.08315428435017953
  (0, 1010)	0.09377912992836293
  (0, 4636)	0.08594898429

In [713]:
tf.get_feature_names()[5000:7000]

['losangelesriots',
 'lose',
 'loser',
 'loses',
 'losing',
 'loss',
 'losses',
 'lossofachild',
 'lossofchild',
 'lossoffamily',
 'lossoffather',
 'lossoffriend',
 'lossofhusband',
 'lossoflover',
 'lossofparents',
 'lossofsenseofreality',
 'lossofwife',
 'lost',
 'lostshoe',
 'lostweekend',
 'lot',
 'lou',
 'louie',
 'louisefletcher',
 'louisgossett',
 'louisiana',
 'louisleterrier',
 'louisprima',
 'louromano',
 'louvre',
 'lovable',
 'love',
 'loveable',
 'loveaffair',
 'loveandromance',
 'loveatfirstsight',
 'loved',
 'lovell',
 'lovely',
 'loveofoneslife',
 'lover',
 'lovers',
 'loves',
 'lovesickness',
 'lovett',
 'loving',
 'low',
 'lower',
 'lows',
 'loyal',
 'loyalties',
 'loyalty',
 'lt',
 'lucbesson',
 'luciaaniello',
 'lucienlaviscount',
 'luck',
 'lucky',
 'lucy',
 'lucyliu',
 'luiscallejo',
 'luisprieto',
 'luke',
 'lukebracey',
 'lukeevans',
 'lunar',
 'lunarmission',
 'lunatic',
 'lunaticasylum',
 'lungcancer',
 'lupin',
 'lupitanyong',
 'lure',
 'lured',
 'lurks',
 'l

In [703]:
tf.get_feature_names().index("stevenspielberg")

7923

In [704]:
tfidf_matrix[:,7926]

<500x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [708]:
genres_indices = []
for i in genres.genre:
    genres_indices.append(tf.get_feature_names().index(i))
print(genres_indices)
print(tfidf_matrix[0,genres_indices[0]])
tfidf_matrix[:,genres_indices] *= 10
print(tfidf_matrix[0,genres_indices[0]])

1.8427983601104443
[3001, 462, 227, 1698, 170, 3015, 7064, 1948, 7282, 8360, 5698, 2506, 8968, 3908, 9052, 3842, 8619, 5678]
18.427983601104444


In [709]:
directors_indices = []
for i in directors:
    directors_indices.append(tf.get_feature_names().index(i))
print(directors_indices)
print(tfidf_matrix[0,directors_indices[0]])
tfidf_matrix[:,directors_indices] *= 3
print(tfidf_matrix[0,directors_indices[0]])

[4736, 6167, 910, 2611, 1558, 8393, 4315, 4306, 1395, 2130, 3254, 5296, 496, 6593, 4426, 1551, 6963, 4522, 2142, 4319, 417, 9048, 7131, 8417, 2052, 8442, 4476, 4316, 4448, 3260, 5837, 5835, 3093, 7027, 3527, 291, 2147, 3631, 7299, 2125, 3432, 7444, 7923, 7132, 3253, 3725, 8207, 4503, 7017, 5424, 7152, 1512, 5135, 6170, 4510, 324, 3375, 5053, 2076, 8383, 7707, 2095, 7443, 5498, 296, 5715, 5132, 196, 8308, 2098, 2140, 3613, 317, 493, 205, 7146, 4758, 7199, 1669, 7428, 6250, 2492, 438, 4425, 5609, 9242, 2788, 1353, 5192, 2256, 7921, 5769, 434, 1557, 4511, 7196, 3433, 4629, 4311, 5553, 1179, 322, 2798, 5295, 2126, 7033, 7023, 4309, 4576, 4297, 4324, 7305, 6253, 6244, 5189, 1230, 3056, 435, 5706, 8192, 8416, 7193, 4365, 1306, 5476, 6287, 7194, 3161, 4266, 4534, 2133, 5054, 1511, 510, 7910, 2257, 3604, 7486, 7008, 308, 3385, 5450, 2178, 5243, 9142, 6951, 9216, 7864, 4493, 1069, 7076, 2794, 4262, 4500, 8857, 1068, 4466, 4472, 4314, 3341, 7052, 5768, 5421, 6696, 4918, 2072, 4514, 8451, 8402, 5

In [710]:
keywords_indices = []
for i in keywords_df.keyword:
    keywords_indices.append(tf.get_feature_names().index(i))
print(keywords_indices)
print(tfidf_matrix[0,keywords_indices[0]])
# tfidf_matrix[:,keywords_indices] *= 10
print(tfidf_matrix[0,keywords_indices[0]])

# erreur -> utiliser une regex
# \(w+\)

ValueError: 'middleearth(tolkien)' is not in list

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

In [ ]:
cosine_sim[0]

In [ ]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
movie='Star Wars'
print("Description of the Movie: ", movie)
print('---------------------------------------------------------------------')
print(movies[movies['title']==movie]['overview'])

In [ ]:
get_recommendations('Star Wars').head(20)

In [ ]:
movie='Toy Story'
print("Description of the Movie: ", movie)
print('---------------------------------------------------------------------')
print(movies[movies['title']==movie]['overview'])

In [ ]:
get_recommendations('Toy Story').head(20)